# CodeNet 데이터 전처리 및 토크나이징

IBM CodeNet 데이터셋을 LM Pretrain용으로 전처리.
전처리 데이터 기반으로 토크나이저 학습

## 주요 파이프라인

1. 코드 전처리: 주석 제거, AST 파서 검증, AST 기반 중복 코드 제거
2. 토크나이즈: WordPiece, Code-specific BPE, Unigram 토크나이저 학습

`code_corpus.parquet`의 문제 ID p00000~p00010 대상으로 테스트 후 전체 데이터에 적용

## 1. Import Libraries

In [1]:
# 기본 라이브러리
import ast
import json
import os
import re
import hashlib
from pathlib import Path
from typing import List, Dict, Any, Optional, Tuple, Set
from collections import defaultdict
import warnings
from tqdm import tqdm
import time

# 데이터 처리
import pandas as pd
import numpy as np

# 토크나이저
from transformers import AutoTokenizer, PreTrainedTokenizerFast
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, trainers, processors
from tokenizers import normalizers
from tokenizers.normalizers import NFD, Lowercase, StripAccents, Sequence
import sentencepiece as spm
from tokenizers.normalizers import Sequence

e:\Dev\NLP\.venv312\Lib\site-packages\transformers\utils\hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


## 2. Load Data

In [2]:
# CodeNet 데이터 경로 설정
data_path = Path("./Project_CodeNet/Project_CodeNet")  # 현재 디렉토리에서 code_corpus.parquet 찾기
parquet_file = data_path / "code_corpus.parquet"

if not parquet_file.exists():
    print(f"Error: {parquet_file} 파일을 찾을 수 없습니다.")
else:
    # 데이터 로딩
    df = pd.read_parquet(parquet_file)

    # 기본 정보 출력
    print(f"전체 데이터 크기: {df.shape}")
    print(f"컬럼들: {df.columns.tolist()}")
    print(f"메모리 사용량: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    
    # 컬럼별 데이터 타입 및 샘플 확인
    print(f"\n컬럼별 정보:")
    for col in df.columns:
        print(f"  {col}: {df[col].dtype}")
        if df[col].dtype == 'object':
            print(f"    샘플: {df[col].iloc[0] if not df[col].isna().all() else 'N/A'}")
    
    # 테스트 대상 필터링 (p00000 ~ p00010) - 0패딩 5자리로 수정
    test_problems = [f"p{i:05d}" for i in range(11)]
    test_df = df[df['problem_id'].isin(test_problems)].copy()
    
    print(f"\n테스트 데이터 크기: {test_df.shape}")
    print(f"테스트 문제들: {sorted(test_df['problem_id'].unique().tolist())}")
    
    # 샘플 데이터 확인
    print("\n샘플 데이터:")
    available_cols = [col for col in ['problem_id', 'language', 'status', 'submission_id', 'user_id'] if col in df.columns]
    print(f"출력 컬럼: {available_cols}")
    print(test_df[available_cols].head())

전체 데이터 크기: (3243424, 11)
컬럼들: ['submission_id', 'problem_id', 'text', 'code_sha1', 'code_bytes', 'language', 'status', 'date', 'user_id', 'n_chars', 'n_lines']
메모리 사용량: 2964.20 MB

컬럼별 정보:
  submission_id: object
    샘플: s429566813
  problem_id: object
    샘플: p00000
  text: object
메모리 사용량: 2964.20 MB

컬럼별 정보:
  submission_id: object
    샘플: s429566813
  problem_id: object
    샘플: p00000
  text: object
    샘플: for i in range (1,10):
    for j in range (1,10):
        print (i,"x",j,"=",i*j)


  code_sha1: object
    샘플: a1020421f9acf6c0ed3a77e3709e0daa36499a46
  code_bytes: int64
  language: object
    샘플: Python
  status: object
    샘플: WA: Presentation Error
  date: int64
  user_id: object
    샘플: u607145912
  n_chars: int64
  n_lines: int64

테스트 데이터 크기: (10655, 11)
테스트 문제들: ['p00000', 'p00001', 'p00002', 'p00003', 'p00004', 'p00005', 'p00006', 'p00007', 'p00008', 'p00009', 'p00010']

샘플 데이터:
출력 컬럼: ['problem_id', 'language', 'status', 'submission_id', 'user_id']
  problem_id languag

## 3. Preprocessing

In [3]:
class CodePreprocessor:
    """
    파이썬 코드 전처리를 위한 클래스
    - 주석 제거 (단일행, 다중행, 독스트링)
    - AST 기반 구조 정규화 및 중복 제거 -> 별도 모듈로 분리
    - 공백 및 들여쓰기 정규화
    """

    def __init__(self, remove_docstrings=True, normalize_whitespace=True):
        self.remove_docstrings = remove_docstrings
        # 메서드 이름과 충돌 방지를 위해 플래그 이름 변경
        self.use_normalize_whitespace = normalize_whitespace

    def remove_comments(self, code: str) -> str:
        """단일행 주석 제거 (문자열 내부 # 보존)"""
        lines = code.split('\n')
        cleaned_lines = []
        
        for line in lines:
            # 문자열 리터럴 내부의 #을 보호하기 위한 간단한 파싱
            in_string = False
            string_char = None
            result = ""
            i = 0
            
            while i < len(line):
                char = line[i]
                
                if not in_string and char in ('"', "'"):
                    # 문자열 시작
                    in_string = True
                    string_char = char
                    result += char
                elif in_string and char == string_char:
                    # 문자열 종료 (이스케이프 확인)
                    if i == 0 or line[i-1] != '\\':
                        in_string = False
                        string_char = None
                    result += char
                elif not in_string and char == '#':
                    # 주석 시작 - 나머지 줄 무시
                    break
                else:
                    result += char
                
                i += 1
            
            cleaned_lines.append(result.rstrip())
        
        return '\n'.join(cleaned_lines)
    
    def remove_docstrings_ast(self, code: str) -> str:
        """AST를 사용한 독스트링 제거"""
        try:
            tree = ast.parse(code)
            
            # 독스트링 제거 로직
            for node in ast.walk(tree):
                if isinstance(node, (ast.FunctionDef, ast.ClassDef, ast.AsyncFunctionDef)):
                    if (node.body and 
                        isinstance(node.body[0], ast.Expr) and 
                        isinstance(node.body[0].value, ast.Constant) and
                        isinstance(node.body[0].value.value, str)):
                        # 독스트링 제거
                        node.body = node.body[1:]
                
                # 모듈 레벨 독스트링 제거
                if isinstance(node, ast.Module):
                    if (node.body and 
                        isinstance(node.body[0], ast.Expr) and 
                        isinstance(node.body[0].value, ast.Constant) and
                        isinstance(node.body[0].value.value, str)):
                        node.body = node.body[1:]
            
            return ast.unparse(tree)
        except:
            # AST 파싱 실패시 원본 반환
            return code
    
    def normalize_whitespace_text(self, code: str) -> str:
        """공백과 들여쓰기 정규화"""
        lines = code.split('\n')
        normalized_lines = []
        
        for line in lines:
            # 탭을 4개 스페이스로 변환
            line = line.expandtabs(4)
            # 행 끝 공백 제거
            line = line.rstrip()
            # 빈 줄이 아닌 경우만 추가
            if line.strip():
                normalized_lines.append(line)
        
        return '\n'.join(normalized_lines)
    
    def get_ast_hash(self, code: str) -> Optional[str]:
        """AST 기반 코드 구조 해시 생성 (중복 감지용)"""
        try:
            tree = ast.parse(code)
            ast_str = ast.dump(tree, include_attributes=False)
            return hashlib.md5(ast_str.encode()).hexdigest()
        except:
            return None
    
    def preprocess(self, code: str, check_duplicates=True) -> Dict[str, Any]:
        """전체 전처리 파이프라인 실행"""
        result = {
            'original_code': code,
            'processed_code': code,
            'ast_hash': None,
            'preprocessing_steps': [],
            'error': None,
            'duplicate_analysis': None
        }
        
        try:
            # 1. 주석 제거
            processed = self.remove_comments(code)
            result['preprocessing_steps'].append('comments_removed')
            
            # 2. 독스트링 제거 (선택적)
            if self.remove_docstrings:
                processed = self.remove_docstrings_ast(processed)
                result['preprocessing_steps'].append('docstrings_removed')
            
            # 3. 공백 정규화 (선택적)
            if self.use_normalize_whitespace:
                processed = self.normalize_whitespace_text(processed)
                result['preprocessing_steps'].append('whitespace_normalized')
            
            result['processed_code'] = processed
            # result['ast_hash'] = ast_hash
            
        except Exception as e:
            result['error'] = str(e)
            result['processed_code'] = code  # 오류시 원본 반환
        
        return result

# 전처리기 인스턴스 생성
preprocessor = CodePreprocessor(remove_docstrings=True)

print("Preprocessor Ready.")

Preprocessor Ready.


### AST 구조 기반 중복 제거기

In [4]:
class ASTDuplicateRemover:
    """
    AST 기반 구조적 코드 중복 제거 클래스 (간소화 버전)
    - 구조적 동일성 검사만 수행 (변수명, 함수명을 정규화하여 로직만 비교)
    - 같은 problem_id 내에서만 중복 검사 수행
    """
    
    def __init__(self, normalize_names=True):
        self.normalize_names = normalize_names
        
    def normalize_ast(self, tree: ast.AST) -> ast.AST:
        """AST를 정규화하여 변수명/함수명 차이를 무시"""
        if not self.normalize_names:
            return tree
            
        # 변수명/함수명을 일반화
        name_counter = {'var': 0, 'func': 0, 'class': 0}
        name_mapping = {}
        
        class NameNormalizer(ast.NodeTransformer):
            def visit_Name(self, node):
                if isinstance(node.ctx, (ast.Store, ast.Load)):
                    if node.id not in name_mapping:
                        if node.id in ['True', 'False', 'None']:  # 내장 상수 보존
                            return node
                        name_mapping[node.id] = f"var_{name_counter['var']}"
                        name_counter['var'] += 1
                    node.id = name_mapping[node.id]
                return node
                
            def visit_FunctionDef(self, node):
                if node.name not in name_mapping:
                    name_mapping[node.name] = f"func_{name_counter['func']}"
                    name_counter['func'] += 1
                node.name = name_mapping[node.name]
                
                # 매개변수 정규화
                for arg in node.args.args:
                    if arg.arg not in name_mapping:
                        name_mapping[arg.arg] = f"var_{name_counter['var']}"
                        name_counter['var'] += 1
                    arg.arg = name_mapping[arg.arg]
                
                return self.generic_visit(node)
                
            def visit_ClassDef(self, node):
                if node.name not in name_mapping:
                    name_mapping[node.name] = f"class_{name_counter['class']}"
                    name_counter['class'] += 1
                node.name = name_mapping[node.name]
                return self.generic_visit(node)
        
        normalizer = NameNormalizer()
        return normalizer.visit(tree)
    
    def get_structural_hash(self, code: str) -> Optional[str]:
        """구조적 해시 생성 (변수명 정규화 후)"""
        try:
            tree = ast.parse(code)
            normalized_tree = self.normalize_ast(tree)
            
            # 위치 정보와 이름을 제외한 구조만 해시
            structure_str = ast.dump(normalized_tree, include_attributes=False)
            return hashlib.sha256(structure_str.encode()).hexdigest()
        except:
            return None
    
    def find_duplicates(self, df: pd.DataFrame, problem_id_col: str = 'problem_id', 
                       code_col: str = 'text') -> Tuple[Set[int], Dict[str, Any]]:
        """구조적 중복 코드를 찾아 제거할 인덱스 반환 (같은 problem_id 내에서만)"""
        duplicates_to_remove = set()
        stats = {
            'total_samples': len(df),
            'structural_groups': {},
            'duplicate_pairs': [],
            'removed_count': 0,
            'problems_processed': 0
        }
        
        print(f"구조적 중복 검사 시작: {len(df)} 개 샘플")
        
        # problem_id별로 그룹화하여 처리
        for problem_id, group in tqdm(df.groupby(problem_id_col), desc="Problem별 구조적 중복 검사"):
            stats['problems_processed'] += 1
            group_indices = group.index.tolist()
            
            # 구조적 중복 검사만 수행
            structural_hashes = {}
            
            for idx in group_indices:
                if idx in duplicates_to_remove:
                    continue
                    
                code = str(group.loc[idx, code_col])
                
                # 구조적 해시 계산
                struct_hash = self.get_structural_hash(code)
                
                if struct_hash and struct_hash in structural_hashes:
                    # 구조적 중복 발견 - 나중에 나온 것을 제거
                    original_idx = structural_hashes[struct_hash]
                    duplicates_to_remove.add(idx)
                    stats['duplicate_pairs'].append((original_idx, idx))
                    
                    if struct_hash not in stats['structural_groups']:
                        stats['structural_groups'][struct_hash] = [original_idx]
                    stats['structural_groups'][struct_hash].append(idx)
                elif struct_hash:  # 유효한 해시인 경우에만 저장
                    structural_hashes[struct_hash] = idx
        
        stats['removed_count'] = len(duplicates_to_remove)
        
        print(f"구조적 중복 검사 완료: {stats['removed_count']} 개 중복 발견")
        print(f"처리된 문제 수: {stats['problems_processed']}")
        print(f"구조적 중복 그룹: {len(stats['structural_groups'])}")
        
        return duplicates_to_remove, stats
    
    def get_basic_stats(self, code: str) -> Dict[str, Any]:
        """기본적인 코드 통계 정보 추출 (로깅용)"""
        try:
            tree = ast.parse(code)
            stats = {
                'num_functions': 0,
                'num_classes': 0,
                'num_lines': len(code.split('\n'))
            }
            
            for node in ast.walk(tree):
                if isinstance(node, ast.FunctionDef):
                    stats['num_functions'] += 1
                elif isinstance(node, ast.ClassDef):
                    stats['num_classes'] += 1
            
            return stats
        except:
            return {'num_functions': 0, 'num_classes': 0, 'num_lines': len(code.split('\n'))}

# AST 중복 제거기 인스턴스 생성 (간소화 버전)
ast_duplicate_remover = ASTDuplicateRemover(normalize_names=True)
print("AST Duplicate Remover Ready.")

AST Duplicate Remover Ready.


### AST 중복검사 실험

In [5]:
# 실제 CodeNet 데이터에 간소화된 중복 제거 적용 테스트
print("=== 실제 CodeNet 데이터 중복 제거 테스트 ===")

# 작은 샘플로 테스트 (p00000~p00010)
test_problems = ['p00000', 'p00001', 'p00002','p00003','p00004','p00005','p00006','p00007','p00008','p00009','p00010']
test_df = df[df['problem_id'].isin(test_problems)].copy()
print(f"테스트 데이터: {len(test_df)}개 샘플")
print(f"문제 분포: {test_df['problem_id'].value_counts().to_dict()}")

# 중복 제거 수행
print("\n중복 제거 수행 중...")
duplicates_to_remove, duplicate_stats = ast_duplicate_remover.find_duplicates(
    test_df, 
    problem_id_col='problem_id', 
    code_col='text'
)

print(f"\n=== 중복 제거 결과 ===")
print(f"원본 샘플 수: {duplicate_stats['total_samples']}")
print(f"제거된 중복 수: {duplicate_stats['removed_count']}")
print(f"남은 샘플 수: {duplicate_stats['total_samples'] - duplicate_stats['removed_count']}")
print(f"중복률: {duplicate_stats['removed_count'] / duplicate_stats['total_samples'] * 100:.1f}%")
print(f"처리된 문제 수: {duplicate_stats['problems_processed']}")
print(f"구조적 중복 그룹 수: {len(duplicate_stats['structural_groups'])}")

# 문제별 중복 현황
print(f"\n=== 문제별 중복 현황 ===")
problem_duplicates = {}
for original_idx, duplicate_idx in duplicate_stats['duplicate_pairs']:
    problem_id = test_df.loc[original_idx, 'problem_id']
    if problem_id not in problem_duplicates:
        problem_duplicates[problem_id] = 0
    problem_duplicates[problem_id] += 1

for problem_id in test_problems:
    original_count = len(test_df[test_df['problem_id'] == problem_id])
    duplicate_count = problem_duplicates.get(problem_id, 0)
    remaining_count = original_count - duplicate_count
    print(f"{problem_id}: {original_count}개 → {remaining_count}개 (중복 {duplicate_count}개 제거)")

# 중복 제거 후 데이터 생성
clean_test_df = test_df.drop(index=duplicates_to_remove)
print(f"\n최종 정리된 데이터: {len(clean_test_df)}개")

=== 실제 CodeNet 데이터 중복 제거 테스트 ===
테스트 데이터: 10655개 샘플
문제 분포: {'p00000': 1626, 'p00001': 1325, 'p00009': 1246, 'p00004': 1240, 'p00002': 1221, 'p00003': 1128, 'p00005': 715, 'p00007': 702, 'p00006': 603, 'p00008': 518, 'p00010': 331}

중복 제거 수행 중...
구조적 중복 검사 시작: 10655 개 샘플


Problem별 구조적 중복 검사:   0%|          | 0/11 [00:00<?, ?it/s]<unknown>:5: SyntaxWarning: invalid escape sequence '\?'
<unknown>:5: SyntaxWarning: invalid escape sequence '\?'
Problem별 구조적 중복 검사:  27%|██▋       | 3/11 [00:00<00:02,  3.73it/s]<unknown>:1: SyntaxWarning: invalid decimal literal
<unknown>:1: SyntaxWarning: invalid decimal literal
<unknown>:1: SyntaxWarning: invalid decimal literal
<unknown>:1: SyntaxWarning: invalid decimal literal
Problem별 구조적 중복 검사:  82%|████████▏ | 9/11 [00:02<00:00,  5.19it/s]<unknown>:5: SyntaxWarning: invalid escape sequence '\?'
<unknown>:17: SyntaxWarning: invalid escape sequence '\?'
<unknown>:5: SyntaxWarning: invalid escape sequence '\?'
<unknown>:17: SyntaxWarning: invalid escape sequence '\?'
Problem별 구조적 중복 검사:  91%|█████████ | 10/11 [00:02<00:00,  2.83it/s]<unknown>:13: SyntaxWarning: invalid escape sequence '\?'
<unknown>:13: SyntaxWarning: invalid escape sequence '\?'
Problem별 구조적 중복 검사: 100%|██████████| 11/11 [00:03<00:00,  3.34it/s]

구조적 중복 검사 완료: 2374 개 중복 발견
처리된 문제 수: 11
구조적 중복 그룹: 833

=== 중복 제거 결과 ===
원본 샘플 수: 10655
제거된 중복 수: 2374
남은 샘플 수: 8281
중복률: 22.3%
처리된 문제 수: 11
구조적 중복 그룹 수: 833

=== 문제별 중복 현황 ===
p00000: 1626개 → 900개 (중복 726개 제거)
p00001: 1325개 → 1017개 (중복 308개 제거)
p00002: 1221개 → 1002개 (중복 219개 제거)
p00003: 1128개 → 895개 (중복 233개 제거)
p00004: 1240개 → 1084개 (중복 156개 제거)
p00005: 715개 → 629개 (중복 86개 제거)
p00006: 603개 → 333개 (중복 270개 제거)
p00007: 702개 → 566개 (중복 136개 제거)
p00008: 518개 → 450개 (중복 68개 제거)
p00009: 1246개 → 1108개 (중복 138개 제거)
p00010: 331개 → 297개 (중복 34개 제거)

최종 정리된 데이터: 8281개


## 토크나이저

In [6]:
class CodeTokenizerFactory:
    """
    코드 전용 토크나이저 팩토리 클래스
    WordPiece, BPE, Unigram 토크나이저를 제공
    """
    
    @staticmethod
    def create_wordpiece_tokenizer(vocab_size=32000, min_frequency=2):
        """WordPiece 토크나이저 생성"""
        tokenizer = Tokenizer(models.WordPiece(unk_token="[UNK]"))
        
        # 코드 전용 정규화 설정
        from tokenizers.normalizers import Sequence
        tokenizer.normalizer = Sequence([
            NFD(),
            StripAccents()
        ])
        
        # 코드 토큰 분리기 (공백, 연산자, 괄호 등)
        tokenizer.pre_tokenizer = pre_tokenizers.Sequence([
            pre_tokenizers.WhitespaceSplit(),
            pre_tokenizers.Punctuation(),
            pre_tokenizers.Digits(individual_digits=True)
        ])
        
        # 특수 토큰 설정
        special_tokens = [
            "[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]",
            "[INDENT]", "[DEDENT]", "[NEWLINE]", "[EOF]"
        ]
        
        trainer = trainers.WordPieceTrainer(
            vocab_size=vocab_size,
            min_frequency=min_frequency,
            special_tokens=special_tokens,
            continuing_subword_prefix="##"
        )
        
        tokenizer.decoder = decoders.WordPiece()
        
        return tokenizer, trainer
    
    @staticmethod
    def create_bpe_tokenizer(vocab_size=32000, min_frequency=2):
        """Code-specific BPE 토크나이저 생성"""
        tokenizer = Tokenizer(models.BPE(unk_token="<unk>"))
        
        # 코드 전용 정규화 설정
        tokenizer.normalizer = Sequence([
            NFD(),
            StripAccents()
        ])
        
        # 코드 구조를 고려한 토큰 분리
        tokenizer.pre_tokenizer = pre_tokenizers.Sequence([
            pre_tokenizers.ByteLevel(add_prefix_space=False),
            pre_tokenizers.Punctuation(),
            pre_tokenizers.Digits(individual_digits=True)
        ])
        
        special_tokens = [
            "<unk>", "<s>", "</s>", "<pad>", "<mask>",
            "<indent>", "<dedent>", "<newline>", "<eof>"
        ]
        
        trainer = trainers.BpeTrainer(
            vocab_size=vocab_size,
            min_frequency=min_frequency,
            special_tokens=special_tokens,
            initial_alphabet=pre_tokenizers.ByteLevel.alphabet()
        )
        
        tokenizer.decoder = decoders.ByteLevel()
        
        return tokenizer, trainer
    
    @staticmethod
    def create_sentencepiece_tokenizer(vocab_size=32000, model_type='unigram'):
        """SentencePiece (Unigram) 토크나이저 생성"""
        return {
            'vocab_size': min(vocab_size, 2000),  # 최대 vocab_size를 2000으로 제한
            'model_type': model_type,
            'pad_id': 0,
            'unk_id': 1, 
            'bos_id': 2,
            'eos_id': 3,
            'pad_piece': '<pad>',
            'unk_piece': '<unk>',
            'bos_piece': '<s>',
            'eos_piece': '</s>',
            'user_defined_symbols': ['<indent>', '<dedent>', '<newline>'],
            'character_coverage': 0.9995,  # 더 보수적인 character coverage
            'input_sentence_size': 1000000,  # 더 작은 입력 크기
            'shuffle_input_sentence': True,
            'max_sentence_length': 4192,  # 최대 문장 길이 제한
            'split_by_whitespace': False  # 코드에 적합하도록 설정
        }

# 토크나이저 팩토리 인스턴스 생성
tokenizer_factory = CodeTokenizerFactory()
print("3 Way Tokenizer Ready")

3 Way Tokenizer Ready


In [ ]:
# ... 상단 동일 (클래스 정의는 이전 셀 수정 반영) ...
class CompletePipelineProcessor:
    """
    전체 CodeNet 데이터 처리 파이프라인
    전처리 → AST 중복 제거 → 다중 토크나이저 훈련 → 최종 데이터셋 생성
    """
    
    def __init__(self, preprocessor, ast_duplicate_remover, tokenizer_factory):
        self.preprocessor = preprocessor
        self.ast_duplicate_remover = ast_duplicate_remover
        self.tokenizer_factory = tokenizer_factory
        self.pipeline_stats = {
            'total_runtime': 0,
            'preprocessing_stats': {},
            'duplicate_removal_stats': {},
            'tokenization_stats': {},
            'final_dataset_stats': {}
        }
    
    def _validate_ast_parseable(self, code: str) -> bool:
        try:
            ast.parse(code)
            return True
        except:
            return False
    
    def process_complete_pipeline(self, df, problem_range, 
                                tokenizers_to_train=['wordpiece', 'bpe', 'unigram'], 
                                vocab_size=32000, output_dir='./output', 
                                experiment_name='codenet_pipeline'):
        start_time = time.time()
        
        print(f"CodeNet 완전 파이프라인 시작: {experiment_name}")
        print(f"대상 문제: {problem_range[0]} ~ {problem_range[-1]} ({len(problem_range)}개 문제)")
        print(f"훈련할 토크나이저: {', '.join(tokenizers_to_train)}")
        os.makedirs(output_dir, exist_ok=True)
        
        # 1단계
        print("\n" + "="*60)
        print("1단계: 데이터 필터링 및 AST 검증")
        print("="*60)
        filtered_df = df[df['problem_id'].isin(problem_range)].copy()
        print(f"문제 범위 필터링: {len(filtered_df):,}개 샘플")
        
        ast_valid_mask = []
        ast_invalid_count = 0
        for idx, row in tqdm(filtered_df.iterrows(), total=len(filtered_df), desc="AST 검증"):
            code = str(row['text'])
            if self._validate_ast_parseable(code):
                ast_valid_mask.append(True)
            else:
                ast_valid_mask.append(False)
                ast_invalid_count += 1
        filtered_df = filtered_df[ast_valid_mask].copy()
        print(f"AST 검증 완료:")
        print(f"  - AST 파싱 불가: {ast_invalid_count:,}개 제거")
        print(f"  - AST 파싱 가능: {len(filtered_df):,}개 유지")
        print(f"  - AST 파싱 성공률: {len(filtered_df)/(len(filtered_df)+ast_invalid_count)*100:.1f}%")
        problem_distribution = filtered_df['problem_id'].value_counts().sort_index()
        print(f"문제별 분포 (AST 검증 후):")
        for problem_id, count in problem_distribution.items():
            print(f"  {problem_id}: {count:,}개")
        self.pipeline_stats['ast_validation_stats'] = {
            'total_samples': len(filtered_df) + ast_invalid_count,
            'ast_valid_samples': len(filtered_df),
            'ast_invalid_samples': ast_invalid_count,
            'ast_success_rate': len(filtered_df)/(len(filtered_df)+ast_invalid_count)*100
        }
        
        # 2단계
        print("\n" + "="*60)
        print("2단계: 코드 전처리")
        print("="*60)
        preprocessing_start = time.time()
        preprocessed_data = []
        preprocessing_errors = 0
        for idx, row in tqdm(filtered_df.iterrows(), total=len(filtered_df), desc="전처리 중"):
            code = str(row['text'])
            result = self.preprocessor.preprocess(code, check_duplicates=False)
            processed_item = {
                'original_index': idx,
                'submission_id': row['submission_id'],
                'user_id': row['user_id'],
                'problem_id': row['problem_id'],
                'language': row['language'],
                'status': row['status'],
                'original_code': code,
                'processed_code': result['processed_code'],
                'preprocessing_error': result['error'],
                'preprocessing_steps': result['preprocessing_steps']
            }
            if result['error'] is not None:
                preprocessing_errors += 1
            preprocessed_data.append(processed_item)
        preprocessing_time = time.time() - preprocessing_start
        print(f"전처리 완료: {len(preprocessed_data):,}개 처리, {preprocessing_errors}개 오류")
        print(f"전처리 시간: {preprocessing_time:.2f}초")
        self.pipeline_stats['preprocessing_stats'] = {
            'total_samples': len(preprocessed_data),
            'errors': preprocessing_errors,
            'success_rate': (len(preprocessed_data) - preprocessing_errors) / len(preprocessed_data) * 100,
            'processing_time': preprocessing_time
        }
        
        # 3단계
        print("\n" + "="*60)
        print("3단계: AST 기반 구조적 중복 제거")
        print("="*60)
        preprocessed_df = pd.DataFrame(preprocessed_data)
        dedup_start = time.time()
        duplicates_to_remove, duplicate_stats = self.ast_duplicate_remover.find_duplicates(
            preprocessed_df, problem_id_col='problem_id', code_col='processed_code')
        dedup_time = time.time() - dedup_start
        clean_df = preprocessed_df.drop(index=duplicates_to_remove)
        print(f"중복 제거 완료:")
        print(f"  - 원본: {len(preprocessed_df):,}개")
        print(f"  - 중복 제거: {len(duplicates_to_remove):,}개 ({len(duplicates_to_remove)/len(preprocessed_df)*100:.1f}%)")
        print(f"  - 최종: {len(clean_df):,}개")
        print(f"  - 처리 시간: {dedup_time:.2f}초")
        self.pipeline_stats['duplicate_removal_stats'] = duplicate_stats
        self.pipeline_stats['duplicate_removal_stats']['processing_time'] = dedup_time
        
        # 4단계
        print("\n" + "="*60)
        print("4단계: 다중 토크나이저 훈련")
        print("="*60)
        tokenization_start = time.time()
        if 'preprocessing_error' not in clean_df.columns:
            clean_df['preprocessing_error'] = pd.Series([None] * len(clean_df), index=clean_df.index)
        clean_df['processed_code'] = clean_df['processed_code'].astype(str)
        non_empty_code = clean_df['processed_code'].str.strip().str.len() > 0
        no_error = clean_df['preprocessing_error'].isna() | (clean_df['preprocessing_error'] == '')
        mask_ok = non_empty_code & no_error
        clean_texts = clean_df.loc[mask_ok, 'processed_code'].tolist()
        print(f"토크나이저 훈련용 텍스트: {len(clean_texts):,}개")
        print("  - clean_df 행 수:", len(clean_df))
        print("  - preprocessing_error가 NA/빈 문자열인 행 수:", int((clean_df['preprocessing_error'].isna() | (clean_df['preprocessing_error'] == '')).sum()))
        print("  - processed_code 비어있지 않은 행 수:", int(non_empty_code.sum()))
        print("  - 최종 선택(mask_ok) 행 수:", int(mask_ok.sum()))
        
        tokenizer_dir = os.path.join(output_dir, 'tokenizers')
        os.makedirs(tokenizer_dir, exist_ok=True)
        tokenizers = {}
        tokenizer_stats = {}
        
        if len(clean_texts) == 0:
            print("훈련용 텍스트가 0개입니다. 토크나이저 훈련 단계를 건너뜁니다.")
        else:
            for tokenizer_type in tokenizers_to_train:
                print(f"\n{tokenizer_type.upper()} 토크나이저 훈련 중...")
                type_start = time.time()
                if tokenizer_type == 'wordpiece':
                    tokenizer, trainer = self.tokenizer_factory.create_wordpiece_tokenizer(vocab_size=vocab_size, min_frequency=2)
                    tokenizer.train_from_iterator(clean_texts, trainer)
                    save_path = os.path.join(tokenizer_dir, f'{experiment_name}_wordpiece_tokenizer.json')
                    tokenizer.save(save_path)
                    sample_text = clean_texts[0]
                    encoding = tokenizer.encode(sample_text)
                    sample_tokens = encoding.tokens
                    vocab_size_actual = tokenizer.get_vocab_size()
                elif tokenizer_type == 'bpe':
                    tokenizer, trainer = self.tokenizer_factory.create_bpe_tokenizer(vocab_size=vocab_size, min_frequency=2)
                    tokenizer.train_from_iterator(clean_texts, trainer)
                    save_path = os.path.join(tokenizer_dir, f'{experiment_name}_bpe_tokenizer.json')
                    tokenizer.save(save_path)
                    sample_text = clean_texts[0]
                    encoding = tokenizer.encode(sample_text)
                    sample_tokens = encoding.tokens
                    vocab_size_actual = tokenizer.get_vocab_size()
                elif tokenizer_type == 'unigram':
                    temp_corpus_file = os.path.join(output_dir, 'temp_corpus.txt')
                    with open(temp_corpus_file, 'w', encoding='utf-8') as f:
                        for text in clean_texts:
                            f.write(text + '\n')
                    max_unigram_vocab = min(vocab_size, max(len(clean_texts) // 2, 200))
                    sp_config = self.tokenizer_factory.create_sentencepiece_tokenizer(vocab_size=max_unigram_vocab, model_type='unigram')
                    model_prefix = os.path.join(tokenizer_dir, f'{experiment_name}_unigram')
                    print(f"  Unigram vocab_size 조정: {vocab_size} → {max_unigram_vocab}")
                    spm.SentencePieceTrainer.train(input=temp_corpus_file, model_prefix=model_prefix, **sp_config)
                    tokenizer = spm.SentencePieceProcessor()
                    tokenizer.load(f'{model_prefix}.model')
                    sample_text = clean_texts[0]
                    sample_tokens = tokenizer.encode_as_pieces(sample_text)
                    vocab_size_actual = tokenizer.get_piece_size()
                    save_path = f'{model_prefix}.model'
                    if os.path.exists(temp_corpus_file):
                        os.remove(temp_corpus_file)
                type_time = time.time() - type_start
                tokenizers[tokenizer_type] = tokenizer
                tokenizer_stats[tokenizer_type] = {
                    'type': tokenizer_type,
                    'vocab_size': vocab_size_actual,
                    'training_time': type_time,
                    'save_path': save_path,
                    'sample_tokens': sample_tokens[:10] if sample_tokens else []
                }
                print(f"{tokenizer_type.upper()} 완료: {type_time:.2f}초, 어휘 크기: {vocab_size_actual:,}")
        total_tokenization_time = time.time() - tokenization_start
        print(f"\n모든 토크나이저 훈련 완료: {total_tokenization_time:.2f}초")
        self.pipeline_stats['tokenization_stats'] = {
            'all_tokenizers': tokenizer_stats,
            'trained_tokenizers': tokenizers_to_train if len(clean_texts) > 0 else [],
            'total_training_time': total_tokenization_time,
            'training_texts': len(clean_texts)
        }
        
        # 5단계 이후 동일 ...
        print("\n" + "="*60)
        print("5단계: 최종 LM 사전훈련 데이터셋 생성 (전처리된 코드만)")
        print("="*60)
        final_start = time.time()
        final_dataset = []
        for _, row in tqdm(clean_df.iterrows(), total=len(clean_df), desc="최종 데이터셋 생성"):
            if not (pd.isna(row['preprocessing_error']) or row['preprocessing_error'] == ''):
                continue
            code = str(row['processed_code'])
            if len(code.strip()) == 0:
                continue
            final_item = {
                'submission_id': row['submission_id'],
                'user_id': row['user_id'],
                'problem_id': row['problem_id'],
                'language': row['language'],
                'status': row['status'],
                'original_index': row['original_index'],
                'original_code': row['original_code'],
                'processed_code': code,
                'code_length': len(code)
            }
            final_dataset.append(final_item)
        final_time = time.time() - final_start
        print(f"최종 데이터셋 생성 완료: {len(final_dataset):,}개 샘플, {final_time:.2f}초")
        print(f"토큰화는 모델 학습시 필요한 토크나이저로 수행")
        print(f"저장된 메타데이터: submission_id, user_id, problem_id, language, status")
        final_df = pd.DataFrame(final_dataset)
        dataset_path = os.path.join(output_dir, f'{experiment_name}_final_dataset.parquet')
        final_df.to_parquet(dataset_path, index=False)
        print(f"최종 데이터셋 저장: {dataset_path}")
        print(f"데이터셋 컬럼: {final_df.columns.tolist()}")
        total_time = time.time() - start_time
        self.pipeline_stats['total_runtime'] = total_time
        self.pipeline_stats['final_dataset_stats'] = {
            'total_samples': len(final_dataset),
            'problems_count': len(final_df['problem_id'].unique()),
            'avg_code_length': final_df['code_length'].mean() if len(final_df) else 0,
            'total_code_length': final_df['code_length'].sum() if len(final_df) else 0,
            'processing_time': final_time
        }
        problem_final_stats = {}
        for problem_id in problem_range:
            problem_data = final_df[final_df['problem_id'] == problem_id]
            original_count = problem_distribution.get(problem_id, 0)
            final_count = len(problem_data)
            problem_final_stats[problem_id] = {
                'original_count': original_count,
                'final_count': final_count,
                'removal_count': original_count - final_count,
                'removal_rate': (original_count - final_count) / original_count * 100 if original_count > 0 else 0,
                'avg_code_length': problem_data['code_length'].mean() if len(problem_data) > 0 else 0,
                'total_code_length': problem_data['code_length'].sum() if len(problem_data) > 0 else 0
            }
        self.pipeline_stats['problem_stats'] = problem_final_stats

        # 통계 저장
        stats_path = os.path.join(output_dir, f'{experiment_name}_pipeline_stats.json')
        with open(stats_path, 'w', encoding='utf-8') as f:
            json.dump(self.pipeline_stats, f, indent=2, ensure_ascii=False, default=str)
        print(f"파이프라인 통계 저장: {stats_path}")

        self._generate_summary_report(output_dir, experiment_name, tokenizers_to_train if len(clean_texts) > 0 else [])
        
        print(f"\n전체 파이프라인 완료")
        print(f"총 실행 시간: {total_time:.2f}초 ({total_time/60:.1f}분)")
        print(f"출력 디렉토리: {output_dir}")
        print(f"훈련된 토크나이저: {', '.join(tokenizers_to_train if len(clean_texts) > 0 else [])}")
        print(f"메타데이터: submission_id, user_id 포함")
        
        return final_df, self.pipeline_stats

    def _generate_summary_report(self, output_dir, experiment_name, tokenizers_trained):
        """요약 리포트 생성"""
        report_path = os.path.join(output_dir, f'{experiment_name}_summary_report.txt')
        
        with open(report_path, 'w', encoding='utf-8') as f:
            f.write(f"CodeNet 파이프라인 처리 리포트\n")
            f.write(f"실험명: {experiment_name}\n")
            f.write(f"처리 시간: {time.strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write("="*60 + "\n\n")
            
            # 전체 요약
            f.write("전체 처리 요약\n")
            f.write("-"*30 + "\n")
            f.write(f"총 실행 시간: {self.pipeline_stats['total_runtime']:.2f}초\n")
            f.write(f"최종 데이터셋 크기: {self.pipeline_stats['final_dataset_stats']['total_samples']:,}개\n")
            f.write(f"처리된 문제 수: {self.pipeline_stats['final_dataset_stats']['problems_count']}개\n")
            f.write(f"총 코드 길이: {self.pipeline_stats['final_dataset_stats']['total_code_length']:,} 문자\n")
            f.write(f"평균 코드 길이: {self.pipeline_stats['final_dataset_stats']['avg_code_length']:.1f} 문자\n\n")
            
            # AST 검증 결과
            if 'ast_validation_stats' in self.pipeline_stats:
                f.write("AST 검증 결과\n")
                f.write("-"*30 + "\n")
                f.write(f"AST 파싱 가능: {self.pipeline_stats['ast_validation_stats']['ast_valid_samples']:,}개\n")
                f.write(f"AST 파싱 불가: {self.pipeline_stats['ast_validation_stats']['ast_invalid_samples']:,}개\n")
                f.write(f"AST 성공률: {self.pipeline_stats['ast_validation_stats']['ast_success_rate']:.1f}%\n\n")
            
            # 단계별 처리 시간
            f.write("단계별 처리 시간\n")
            f.write("-"*30 + "\n")
            f.write(f"전처리: {self.pipeline_stats['preprocessing_stats']['processing_time']:.2f}초\n")
            f.write(f"중복 제거: {self.pipeline_stats['duplicate_removal_stats']['processing_time']:.2f}초\n")
            f.write(f"토크나이저 훈련: {self.pipeline_stats['tokenization_stats']['total_training_time']:.2f}초\n")
            f.write(f"최종 데이터셋 생성: {self.pipeline_stats['final_dataset_stats']['processing_time']:.2f}초\n\n")
            
            # 토크나이저 비교
            f.write("토크나이저 훈련 결과\n")
            f.write("-"*30 + "\n")
            f.write(f"훈련된 토크나이저: {', '.join(tokenizers_trained)}\n")
            for name, stats in self.pipeline_stats['tokenization_stats']['all_tokenizers'].items():
                f.write(f"{name}: 어휘 {stats['vocab_size']:,}개, 훈련 {stats['training_time']:.2f}초\n")
            f.write("\n")
            
            # 중복 제거 결과
            f.write("중복 제거 결과\n")
            f.write("-"*30 + "\n")
            f.write(f"제거된 중복: {self.pipeline_stats['duplicate_removal_stats']['removed_count']:,}개\n")
            f.write(f"중복률: {self.pipeline_stats['duplicate_removal_stats']['removed_count'] / self.pipeline_stats['duplicate_removal_stats']['total_samples'] * 100:.1f}%\n")
            f.write(f"구조적 중복 그룹: {len(self.pipeline_stats['duplicate_removal_stats']['structural_groups']):,}개\n\n")
            
            # 문제별 통계
            f.write("문제별 처리 결과\n")
            f.write("-"*30 + "\n")
            for problem_id, stats in self.pipeline_stats['problem_stats'].items():
                f.write(f"{problem_id}: {stats['original_count']:,} → {stats['final_count']:,} ")
                f.write(f"(-{stats['removal_count']:,}, {stats['removal_rate']:.1f}% 제거)\n")
            
            # 사용 안내
            f.write("\n토크나이저 사용 안내\n")
            f.write("-"*30 + "\n")
            f.write("최종 데이터셋에는 전처리된 코드만 저장되어 있습니다.\n")
            f.write("모델 학습시 원하는 토크나이저를 로드하여 토큰화를 수행하세요.\n")
            for name, stats in self.pipeline_stats['tokenization_stats']['all_tokenizers'].items():
                f.write(f"- {name}: {stats['save_path']}\n")
            
            f.write("\n메타데이터 정보\n")
            f.write("-"*30 + "\n")
            f.write("최종 데이터셋에는 다음 메타데이터가 포함됩니다:\n")
            f.write("- submission_id: 제출 고유 ID\n")
            f.write("- user_id: 사용자 고유 ID\n")
            f.write("- problem_id: 문제 고유 ID\n")
            f.write("- language: 프로그래밍 언어\n")
            f.write("- status: 제출 결과 상태\n")
            
        print(f"요약 리포트 저장: {report_path}")


# 통합 파이프라인 프로세서 인스턴스 생성
pipeline_processor = CompletePipelineProcessor(
    preprocessor=preprocessor,
    ast_duplicate_remover=ast_duplicate_remover,
    tokenizer_factory=tokenizer_factory
)

In [ ]:
# p00000~p00010 샘플 실험 진행
print("=== p00000~p00010 샘플 실험 진행 (AST 검증 + 메타데이터 유지) ===")

# 실험 설정
sample_problems = [f"p{i:05d}" for i in range(11)]  # p00000 ~ p00010
experiment_name = "codenet_sample_p00000_p00010_enhanced"
output_dir = "./pipeline_output_enhanced"

print(f"실험 대상: {sample_problems}")
print(f"실험명: {experiment_name}")
print(f"출력 디렉토리: {output_dir}")

# 샘플 실험 실행 (새로운 기능 포함)
sample_final_df, sample_stats = pipeline_processor.process_complete_pipeline(
    df=df,
    problem_range=sample_problems,
    tokenizers_to_train=['wordpiece', 'bpe', 'unigram'],  # 모든 토크나이저 훈련
    vocab_size=16000,  # 샘플 실험용 작은 어휘 크기
    output_dir=output_dir,
    experiment_name=experiment_name
)

print(f"\n 샘플 실험 완료!")
print(f" 결과 요약:")
print(f"  - 최종 데이터셋: {len(sample_final_df):,}개")
print(f"  - 평균 코드 길이: {sample_final_df['code_length'].mean():.1f} 문자")
print(f"  - 총 코드 길이: {sample_final_df['code_length'].sum():,} 문자")
print(f"  - 총 처리 시간: {sample_stats['total_runtime']:.2f}초")
print(f"  - 훈련된 토크나이저: {', '.join(sample_stats['tokenization_stats']['trained_tokenizers'])}")
print(f"  - 토크나이저 훈련 시간: {sample_stats['tokenization_stats']['total_training_time']:.2f}초")

# AST 검증 결과
if 'ast_validation_stats' in sample_stats:
    ast_stats = sample_stats['ast_validation_stats']
    print(f"\n AST 검증 결과:")
    print(f"  - AST 파싱 가능: {ast_stats['ast_valid_samples']:,}개")
    print(f"  - AST 파싱 불가: {ast_stats['ast_invalid_samples']:,}개")
    print(f"  - AST 성공률: {ast_stats['ast_success_rate']:.1f}%")

# 메타데이터 확인
print(f"\n 저장된 메타데이터:")
print(f"  - 데이터셋 컬럼: {sample_final_df.columns.tolist()}")
print(f"  - submission_id 샘플: {sample_final_df['submission_id'].iloc[0]}")
print(f"  - user_id 샘플: {sample_final_df['user_id'].iloc[0]}")
print(f"  - 고유 사용자 id: {sample_final_df['user_id'].nunique():,}명")
print(f"  - 고유 제출 id: {sample_final_df['submission_id'].nunique():,}개")

=== p00000~p00010 샘플 실험 재진행 (AST 검증 + 메타데이터 유지) ===
실험 대상: ['p00000', 'p00001', 'p00002', 'p00003', 'p00004', 'p00005', 'p00006', 'p00007', 'p00008', 'p00009', 'p00010']
실험명: codenet_sample_p00000_p00010_enhanced
출력 디렉토리: ./pipeline_output_enhanced
CodeNet 완전 파이프라인 시작: codenet_sample_p00000_p00010_enhanced
대상 문제: p00000 ~ p00010 (11개 문제)
훈련할 토크나이저: wordpiece, bpe, unigram

1단계: 데이터 필터링 및 AST 검증
문제 범위 필터링: 10,655개 샘플


AST 검증:  40%|████      | 4276/10655 [00:00<00:00, 7872.96it/s]<unknown>:1: SyntaxWarning: invalid decimal literal
<unknown>:1: SyntaxWarning: invalid decimal literal
<unknown>:1: SyntaxWarning: invalid decimal literal
<unknown>:1: SyntaxWarning: invalid decimal literal
AST 검증:  83%|████████▎ | 8817/10655 [00:01<00:00, 7232.93it/s]<unknown>:5: SyntaxWarning: invalid escape sequence '\?'
<unknown>:17: SyntaxWarning: invalid escape sequence '\?'
AST 검증:  90%|████████▉ | 9551/10655 [00:01<00:00, 6374.82it/s]<unknown>:5: SyntaxWarning: invalid escape sequence '\?'
<unknown>:17: SyntaxWarning: invalid escape sequence '\?'
AST 검증: 100%|██████████| 10655/10655 [00:01<00:00, 6534.76it/s]
<unknown>:13: SyntaxWarning: invalid escape sequence '\?'
AST 검증: 100%|██████████| 10655/10655 [00:01<00:00, 6534.76it/s]


AST 검증 완료:
  - AST 파싱 불가: 3,288개 제거
  - AST 파싱 가능: 7,367개 유지
  - AST 파싱 성공률: 69.1%
문제별 분포 (AST 검증 후):
  p00000: 1,138개
  p00001: 949개
  p00002: 804개
  p00003: 841개
  p00004: 798개
  p00005: 509개
  p00006: 442개
  p00007: 540개
  p00008: 325개
  p00009: 809개
  p00010: 212개

2단계: 코드 전처리


전처리 중:  40%|████      | 2947/7367 [00:00<00:01, 3686.91it/s]<unknown>:1: SyntaxWarning: invalid decimal literal
<unknown>:1: SyntaxWarning: invalid decimal literal
<unknown>:1: SyntaxWarning: invalid decimal literal
<unknown>:1: SyntaxWarning: invalid decimal literal
전처리 중:  89%|████████▊ | 6528/7367 [00:02<00:00, 2593.32it/s]<unknown>:5: SyntaxWarning: invalid escape sequence '\?'
<unknown>:17: SyntaxWarning: invalid escape sequence '\?'
전처리 중:  92%|█████████▏| 6806/7367 [00:02<00:00, 2352.61it/s]<unknown>:5: SyntaxWarning: invalid escape sequence '\?'
<unknown>:17: SyntaxWarning: invalid escape sequence '\?'
전처리 중: 100%|██████████| 7367/7367 [00:02<00:00, 2686.12it/s]


전처리 완료: 7,367개 처리, 0개 오류
전처리 시간: 2.74초

3단계: AST 기반 구조적 중복 제거
구조적 중복 검사 시작: 7367 개 샘플


Problem별 구조적 중복 검사: 100%|██████████| 11/11 [00:02<00:00,  4.59it/s]



구조적 중복 검사 완료: 2377 개 중복 발견
처리된 문제 수: 11
구조적 중복 그룹: 835
중복 제거 완료:
  - 원본: 7,367개
  - 중복 제거: 2,377개 (32.3%)
  - 최종: 4,990개
  - 처리 시간: 2.40초

4단계: 다중 토크나이저 훈련
토크나이저 훈련용 텍스트: 4,990개
  - clean_df 행 수: 4990
  - preprocessing_error가 NA/빈 문자열인 행 수: 4990
  - processed_code 비어있지 않은 행 수: 4990
  - 최종 선택(mask_ok) 행 수: 4990

🔤 WORDPIECE 토크나이저 훈련 중...
WORDPIECE 완료: 0.13초, 어휘 크기: 2,188

🔤 BPE 토크나이저 훈련 중...
BPE 완료: 0.23초, 어휘 크기: 2,782

🔤 UNIGRAM 토크나이저 훈련 중...
  Unigram vocab_size 조정: 16000 → 2495
UNIGRAM 완료: 1.38초, 어휘 크기: 2,000

모든 토크나이저 훈련 완료: 1.74초

5단계: 최종 LM 사전훈련 데이터셋 생성 (전처리된 코드만)
BPE 완료: 0.23초, 어휘 크기: 2,782

🔤 UNIGRAM 토크나이저 훈련 중...
  Unigram vocab_size 조정: 16000 → 2495
UNIGRAM 완료: 1.38초, 어휘 크기: 2,000

모든 토크나이저 훈련 완료: 1.74초

5단계: 최종 LM 사전훈련 데이터셋 생성 (전처리된 코드만)


최종 데이터셋 생성: 100%|██████████| 4990/4990 [00:00<00:00, 24447.24it/s]

최종 데이터셋 생성 완료: 4,990개 샘플, 0.21초
토큰화는 모델 학습시 필요한 토크나이저로 수행
저장된 메타데이터: submission_id, user_id, problem_id, language, status
최종 데이터셋 저장: ./pipeline_output_enhanced\codenet_sample_p00000_p00010_enhanced_final_dataset.parquet
데이터셋 컬럼: ['submission_id', 'user_id', 'problem_id', 'language', 'status', 'original_index', 'original_code', 'processed_code', 'code_length']
파이프라인 통계 저장: ./pipeline_output_enhanced\codenet_sample_p00000_p00010_enhanced_pipeline_stats.json
요약 리포트 저장: ./pipeline_output_enhanced\codenet_sample_p00000_p00010_enhanced_summary_report.txt

전체 파이프라인 완료
총 실행 시간: 8.92초 (0.1분)
출력 디렉토리: ./pipeline_output_enhanced
훈련된 토크나이저: wordpiece, bpe, unigram
메타데이터: submission_id, user_id 포함

 샘플 실험 완료!
 결과 요약:
  - 최종 데이터셋: 4,990개
  - 평균 코드 길이: 274.9 문자
  - 총 코드 길이: 1,371,871 문자
  - 총 처리 시간: 8.92초
  - 훈련된 토크나이저: wordpiece, bpe, unigram
  - 토크나이저 훈련 시간: 1.74초

🔍 AST 검증 결과:
  - AST 파싱 가능: 7,367개
  - AST 파싱 불가: 3,288개
  - AST 성공률: 69.1%

 저장된 메타데이터:
  - 데이터셋 컬럼: ['submission_id', 'user_id', 'pr

In [18]:
# 전체 데이터셋 대상 파이프라인 실행 옵션
print("\n" + "="*80)
print("전체 CodeNet 데이터 파이프라인 실행")
print("="*80)

# 전체 CodeNet 데이터 로드
def load_full_codenet_data():
    """전체 CodeNet 데이터를 로드"""
    # 가능한 경로들 시도
    possible_paths = [
        './Project_CodeNet/Project_CodeNet/code_corpus.parquet', # 상위의 하위 디렉토리
    ]
    
    print("전체 CodeNet 데이터 로드...")
    
    for i, path in enumerate(possible_paths, 1):
        print(f"  {i}. 경로 확인: {path}")
        if Path(path).exists():
            try:
                full_data = pd.read_parquet(path)
                print(f"  데이터 로드 성공: {len(full_data):,}개 샘플")
                return full_data
            except Exception as e:
                print(f"  로드 실패: {e}")
        else:
            print(f" 파일 없음")
    
    return None

# 전체 데이터 로드 시도
full_data = load_full_codenet_data()

if full_data is not None:
    # 전체 데이터 정보
    all_problems = sorted(full_data['problem_id'].unique())
    python_data = full_data[full_data['language'] == 'Python']
    
    print(f"\n 전체 CodeNet 데이터 정보:")
    print(f"  - 전체 문제 수: {len(all_problems):,}개")
    print(f"  - 전체 샘플 수: {len(full_data):,}개")
    print(f"  - Python 샘플 수: {len(python_data):,}개")
    print(f"  - 메모리 사용량: {full_data.memory_usage(deep=True).sum() / 1024**3:.2f} GB")
    
    # 언어별 분포
    print(f"\n 상위 언어별 분포:")
    lang_dist = full_data['language'].value_counts().head(10)
    for lang, count in lang_dist.items():
        print(f"  - {lang}: {count:,}개 ({count/len(full_data)*100:.1f}%)")
    
    # 전체 실행 시 예상 시간 계산 (샘플 실험 결과 기반)
    if 'result' in locals() and 'total_runtime' in result:
        sample_processing_rate = result['final_count'] / result['total_runtime']
        estimated_full_time = len(python_data) / sample_processing_rate
        print(f"\n 예상 처리 시간:")
        print(f"  - Python 데이터 기준: {estimated_full_time:.0f}초 ({estimated_full_time/3600:.1f}시간)")
        print(f"  - 샘플 처리 속도: {sample_processing_rate:.1f}개/초")
    else:
        print(f"\n 예상 처리 시간: unknown (샘플 처리 속도 정보 없음)")
    
    
    # 전체 실행 여부 선택
    run_full_pipeline = True  # ⚠️ 전체 파이프라인 실행 ⚠️
    
    print(f"\n💡 전체 파이프라인 실행: {'활성화' if run_full_pipeline else '비활성화'}")
    
    if run_full_pipeline:
        print("\n" + "="*60)
        print("전체 CodeNet 데이터 파이프라인 시작")
        print("="*60)
        
        # 전체 실험 설정
        full_experiment_name = "codenet_full_python_dataset_2"
        full_output_dir = "./pipeline_output_full_2"
        
        # 기존에 생성된 파이프라인 프로세서 사용
        if 'pipeline_processor' not in locals():
            print("pipeline_processor not ready.")
        else:
            # 전체 파이프라인 실행 (Python 코드만)
            print(f"처리 대상: Python 언어 {len(python_data):,}개 샘플")
            print(f"출력 dir: {full_output_dir}")
            
            import time
            start_time = time.time()
            
            # 전체 문제 리스트 생성 (모든 문제 처리)
            full_problem_list = sorted(full_data['problem_id'].unique())
            print(f"전체 문제 범위: {len(full_problem_list):,}개 문제 (모든 문제)")
            
            try:
                full_final_df, full_stats = pipeline_processor.process_complete_pipeline(
                    df=full_data,
                    problem_range=full_problem_list,  # 모든 문제 처리
                    tokenizers_to_train=['wordpiece', 'bpe', 'unigram'],  # 모든 토크나이저
                    vocab_size=32000,  # 전체 데이터용 큰 어휘 크기
                    output_dir=full_output_dir,
                    experiment_name=full_experiment_name
                )
                
                end_time = time.time()
                total_time = end_time - start_time
                
                print(f"\n" + "="*60)
                print(" 전체 파이프라인 완료")
                print("="*60)
                print(f"최종 결과:")
                print(f"  - 최종 데이터셋: {len(full_final_df):,}개")
                print(f"  - 평균 코드 길이: {full_final_df['code_length'].mean():.1f} 문자")
                print(f"  - 총 코드 길이: {full_final_df['code_length'].sum():,} 문자")
                print(f"  - 총 처리 시간: {total_time:.2f}초 ({total_time/3600:.2f}시간)")
                print(f"  - 처리 속도: {len(full_final_df)/total_time:.1f}개/초")
                
                # AST 검증 결과
                if 'ast_validation_stats' in full_stats:
                    ast_stats = full_stats['ast_validation_stats']
                    print(f"\n AST 검증 결과:")
                    print(f"  - AST 파싱 가능: {ast_stats['ast_valid_samples']:,}개")
                    print(f"  - AST 파싱 불가: {ast_stats['ast_invalid_samples']:,}개")
                    print(f"  - AST 성공률: {ast_stats['ast_success_rate']:.1f}%")
                
                # 중복 제거 효과
                if 'duplicate_removal_stats' in full_stats:
                    dup_stats = full_stats['duplicate_removal_stats']
                    print(f"\n 중복 제거 효과:")
                    # 안전하게 키 확인 후 출력
                    if 'removed_count' in dup_stats:
                        print(f"  - 제거된 중복: {dup_stats['removed_count']:,}개")
                    # duplicate_rate 대신 계산된 값 사용
                    if 'removed_count' in dup_stats and 'total_samples' in dup_stats:
                        duplicate_rate = (dup_stats['removed_count'] / dup_stats['total_samples']) * 100
                        print(f"  - 중복률: {duplicate_rate:.1f}%")
                    elif 'duplicate_rate' in dup_stats:
                        print(f"  - 중복률: {dup_stats['duplicate_rate']:.1f}%")
                    if 'total_groups' in dup_stats:
                        print(f"  - 구조적 중복 그룹: {dup_stats['total_groups']:,}개")
                    
                    # 디버깅: 사용 가능한 키 출력
                    print(f"   중복 제거 통계 키: {list(dup_stats.keys())}")
                
                # 토크나이저 결과
                if 'tokenization_stats' in full_stats:
                    tok_stats = full_stats['tokenization_stats']
                    print(f"\n 토크나이저 훈련 결과:")
                    for name, info in tok_stats['all_tokenizers'].items():
                        print(f"  - {name}: 어휘 {info['vocab_size']:,}개, 훈련 {info['training_time']:.2f}초")
                
                print(f"\n 결과 저장 위치: {full_output_dir}")
                print(f" 최종 데이터셋: {full_output_dir}/{full_experiment_name}_final_dataset.parquet")
                print(f" 토크나이저: {full_output_dir}/tokenizers/")
                
            except Exception as e:
                print(f"\n 파이프라인 실행 중 오류 발생:")
                print(f"  오류 내용: {str(e)}")
                print(f"  오류 타입: {type(e).__name__}")
                import traceback
                traceback.print_exc()
        
    else:
        print(f"\n전체 실행 비활성화")

else:
    print("\nCodeNet data not ready")


전체 CodeNet 데이터 파이프라인 실행
전체 CodeNet 데이터 로드...
  1. 경로 확인: ./Project_CodeNet/Project_CodeNet/code_corpus.parquet
  데이터 로드 성공: 3,243,424개 샘플
  데이터 로드 성공: 3,243,424개 샘플

 전체 CodeNet 데이터 정보:
  - 전체 문제 수: 3,234개
  - 전체 샘플 수: 3,243,424개
  - Python 샘플 수: 3,243,424개

 전체 CodeNet 데이터 정보:
  - 전체 문제 수: 3,234개
  - 전체 샘플 수: 3,243,424개
  - Python 샘플 수: 3,243,424개
  - 메모리 사용량: 2.89 GB

 상위 언어별 분포:
  - Python: 3,243,424개 (100.0%)

 예상 처리 시간: unknown (샘플 처리 속도 정보 없음)

💡 전체 파이프라인 실행: 활성화

전체 CodeNet 데이터 파이프라인 시작
처리 대상: Python 언어 3,243,424개 샘플
출력 dir: ./pipeline_output_full_2
전체 문제 범위: 3,234개 문제 (모든 문제)
CodeNet 완전 파이프라인 시작: codenet_full_python_dataset_2
대상 문제: p00000 ~ p04052 (3234개 문제)
훈련할 토크나이저: wordpiece, bpe, unigram

1단계: 데이터 필터링 및 AST 검증
  - 메모리 사용량: 2.89 GB

 상위 언어별 분포:
  - Python: 3,243,424개 (100.0%)

 예상 처리 시간: unknown (샘플 처리 속도 정보 없음)

💡 전체 파이프라인 실행: 활성화

전체 CodeNet 데이터 파이프라인 시작
처리 대상: Python 언어 3,243,424개 샘플
출력 dir: ./pipeline_output_full_2
전체 문제 범위: 3,234개 문제 (모든 문제)
CodeNet 완전 파이프라인 시작: code

AST 검증:   0%|          | 4150/3243424 [00:01<10:21, 5215.61it/s]<unknown>:1: SyntaxWarning: invalid decimal literal
<unknown>:1: SyntaxWarning: invalid decimal literal
<unknown>:1: SyntaxWarning: invalid decimal literal
<unknown>:1: SyntaxWarning: invalid decimal literal
AST 검증:   0%|          | 8951/3243424 [00:02<08:33, 6300.46it/s]<unknown>:5: SyntaxWarning: invalid escape sequence '\?'
<unknown>:17: SyntaxWarning: invalid escape sequence '\?'
AST 검증:   0%|          | 9606/3243424 [00:02<09:15, 5822.13it/s]<unknown>:5: SyntaxWarning: invalid escape sequence '\?'
<unknown>:17: SyntaxWarning: invalid escape sequence '\?'
AST 검증:   0%|          | 10212/3243424 [00:02<09:40, 5571.21it/s]<unknown>:13: SyntaxWarning: invalid escape sequence '\?'
<unknown>:13: SyntaxWarning: invalid escape sequence '\?'
<unknown>:4: SyntaxWarning: invalid decimal literal
AST 검증:   0%|          | 12593/3243424 [00:02<10:13, 5269.57it/s]<unknown>:2: SyntaxWarning: invalid escape sequence '\w'
<unknown>:2: Sy

KeyboardInterrupt: 